# Histogram of power of theta in BLA during REM

In [5]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts

In [6]:
%matplotlib qt

In [7]:
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.facecolor'] = '#282C34'
mpl.rcParams['axes.facecolor'] = '#282C34'

In [8]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [9]:
states = bk.load.states()
run = scipy.io.loadmat(bk.load.session + '-TrackRunTimes.mat')['trackruntimes']
run = nts.IntervalSet(np.round(run[:,0],decimals = 1),np.round(run[:,1],decimals = 1),time_units='s')
theta = {'Rem':states['Rem'],'run':run,'wake':states['wake']}

In [17]:
plt.figure()
plt.suptitle('Theta PhaseShift')
for i,state in enumerate(theta,1):
    
    lfp_hpc = bk.load.lfp_in_intervals(166,23,theta[state])
    lfp_bla = bk.load.lfp_in_intervals(166,105,theta[state])
    
    filt_hpc = bk.signal.passband(lfp_hpc,4,8)
    filt_bla = bk.signal.passband(lfp_bla,4,8)
    
    power_hpc,phase_hpc = bk.signal.hilbert(filt_hpc,deg = True)
    power_bla,phase_bla = bk.signal.hilbert(filt_bla,deg = True)
    
    plt.subplot(2,3,i)
    plt.hist2d(phase_hpc.values+180,phase_bla.values+180,100)
    plt.title(f'Theta Phase shift during {state}')
    plt.xlabel('hpc_phase')
    plt.ylabel('bla_phase')

In [19]:
# plt.figure()
# plt.suptitle('Spectrograms')

for i,state in enumerate(theta,4):
    
    lfp_hpc = bk.load.lfp_in_intervals(166,23,theta[state])
    lfp_bla = bk.load.lfp_in_intervals(166,105,theta[state])
    
    f,spec_hpc = scipy.signal.welch(lfp_hpc,fs = 1250,nperseg=10000,noverlap=9000)
    f,spec_bla = scipy.signal.welch(lfp_bla,fs = 1250,nperseg=10000,noverlap=9000)

    plt.subplot(2,3,i)
    plt.title(f'Power Spectrum during {state}')
    plt.plot(f,spec_hpc)
    plt.plot(f,spec_bla)
    plt.xlim(0,20)
    plt.ylim(0,100000)
    plt.legend(['Hpc','BLA'])

In [36]:
t = np.arange(0,len(lfp_hpc),1)/1250

lfp_bla = bk.load.lfp_in_intervals(166,105,theta['run'])
lfp_hpc= bk.load.lfp_in_intervals(166,13,theta['run'])
plt.figure()
s = plt.subplot(2,1,1)
plt.plot(t,lfp_bla.values, c = 'azure')
plt.plot(t,bk.signal.passband(lfp_bla,4,8).values,c = 'r')
plt.subplot(2,1,2,sharex=s,sharey = s)
plt.plot(t,lfp_hpc.values, c = 'azure')
plt.plot(t,bk.signal.passband(lfp_hpc,4,8).values,c = 'r')